In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["HTTPS_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	add_myscale_retrieval
LLMs:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


In [3]:
import yaml
from retrieval_qa_benchmark.utils.config import load
# config = load(open("../config/mmlu-llama2-remote-retrieval-2staged.yaml"))
config = load(open("../config/mmlu-llama2-remote-retrieval-base.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 6745.40it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fangruil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/fangruil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


load dataset...


Resolving data files:   0%|          | 0/253 [00:00<?, ?it/s]

load index...
load mpnet model...
load Colbert model...
entry_id='0' chain={'0': AddMyScaleRetrievalTransform(children=(None, None), sep_chr='\n', prompt_prefix='Context:', num_filtered=100, num_selected=5, with_title=True, rank_dict={'mpnet': 30, 'bm25': 40}, index_path='/mnt/workspaces/yongqij/evals/data/indexes/Cohere_mpnet/IVFSQ_L2.index', embedding_name='paraphrase-multilingual-mpnet-base-v2', dataset_name=['Cohere/wikipedia-22-12-en-embeddings'])}


In [5]:
import json
from tqdm import tqdm
data = []
for i in tqdm(range(len(evaluator.dataset))):
    data.append(evaluator.transform(evaluator.dataset[i]))
with open('old_aligned.jsonl', 'w') as f:
    f.write('\n'.join([json.dumps(d.model_dump()) for d in data]))
# evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
# data

100%|██████████| 324/324 [04:37<00:00,  1.17it/s]


In [4]:
print(evaluator.llm.convert_record(data))

The following are multiple choice questions (with answers) with context:

Context:
[1] Minoan art | It was influenced by the neighbouring cultures of Ancient Egypt and the ancient Near East, which had produced sophisticated urban art for much longer, but the character of the small but wealthy mercantile Minoan cities was very different, with little evidence of large temple-based religion, monarchs, or warfare, and "all the imaginative power and childlike freshness of a very young culture". All these aspects of the Minoan culture remain rather mysterious. Sinclair Hood described an "essential quality of the finest Minoan art, the ability to create an atmosphere of movement and life although following a set of highly formal conventions".
[2] Minoan civilization | Apart from the abundant local agriculture, the Minoans were also a mercantile people who engaged significantly in overseas trade, and at their peak may well have had a dominant position in international trade over much of the Me

In [5]:
print(str(PROFILER))
PROFILER.clear()

	======== Profile Result ========
transform.base.profile                            	Count:     344	AvgTime:   34.21 ms
dataset.mmlu.profile                              	Count:      10	AvgTime:    0.01 ms
database.FaissSearch.emb_filter.profile           	Count:      10	AvgTime:  394.40 ms
transform.Faiss.profile                           	Count:      10	AvgTime:  394.76 ms
database.RerankSearcher.stage2_search.profile     	Count:      10	AvgTime:  774.70 ms
transform.RRFHybrid.profile                       	Count:      10	AvgTime:  780.87 ms
transform.TransformChain.profile                  	Count:      10	AvgTime: 1175.91 ms



In [ ]:
matched = evaluator()

In [ ]:
print(str(PROFILER))

In [ ]:
with open("results.with-retrieval.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))

In [6]:
import json
with open('old_aligned.jsonl') as f1:
    old_result = [json.loads(l) for l in f1.readlines()]

with open('new.jsonl') as f2:
    new_result = [json.loads(l) for l in f2.readlines()]

In [7]:
from hashlib import sha256
cnt = 0
for r1, r2 in zip(old_result, new_result):
    c1 = set(map(lambda x: sha256(x.encode('utf-8')).hexdigest(), r1['context']))
    c2 = set(map(lambda x: sha256(x.encode('utf-8')).hexdigest(), r2['context']))
    cnt += len(c2.intersection(c1))
print(cnt / len(new_result) / 5)

1.0
